In [59]:
# Imports
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession

import json
import os

import mysql.connector
from enum import Enum

import pyspark.sql.functions as func



In [13]:
def parse_JSON_list(filename):
    branch_list = []

    with open(filename) as f:
        data = json.load(f)

    for i in data:
        branch_list.append(i)

    f.close()

    return branch_list



In [14]:
def process_directory(directoryName):
    parsed_list = {}
    for filename in os.listdir(directoryName):
        f = os.path.join(directoryName, filename)
        if os.path.isfile(f):
            table_name = filename.split(".")[0].upper()
            parsed_list[table_name] = parse_JSON_list(f)
        else:
            print("Unable to find", f)
    return parsed_list

In [15]:
class TABLE_NAME(Enum):
    CDW_SAPP_CREDIT = "CDW_SAPP_CREDIT"
    CDW_SAPP_BRANCH = "CDW_SAPP_BRANCH"
    CDW_SAPP_CUSTOMER = "CDW_SAPP_CUSTOMER"

In [33]:
# create the SparkSession
spark = SparkSession.builder.appName('Bank_Analysis').getOrCreate()

parsed_lists = process_directory("json")

for name in parsed_lists:
    match name:
        case "CDW_SAPP_CREDIT":
            rdd_credit = spark.createDataFrame(parsed_lists["CDW_SAPP_CREDIT"])
        case "CDW_SAPP_BRANCH":
            rdd_branch = spark.createDataFrame(parsed_lists["CDW_SAPP_BRANCH"])
        case "CDW_SAPP_CUSTOMER":
            rdd_customer = spark.createDataFrame(parsed_lists["CDW_SAPP_CUSTOMER"])
        case _:
            print(name, "was not a recognized JSON file")




In [60]:
rdd_customert = rdd_customer.withColumn("FULL_STREET_ADDRESS", func.concat(rdd_customer["STREET_NAME"], func.lit(", ") , rdd_customer["APT_NO"]))

rdd_customert.show()

+------+----------------+------------+-------------+--------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+--------------------+
|APT_NO|  CREDIT_CARD_NO|   CUST_CITY| CUST_COUNTRY|          CUST_EMAIL|CUST_PHONE|CUST_STATE|CUST_ZIP|FIRST_NAME|LAST_NAME|        LAST_UPDATED|MIDDLE_NAME|      SSN|      STREET_NAME| FULL_STREET_ADDRESS|
+------+----------------+------------+-------------+--------------------+----------+----------+--------+----------+---------+--------------------+-----------+---------+-----------------+--------------------+
|   656|4210653310061055|     Natchez|United States| AHooper@example.com|   1237818|        MS|   39120|      Alec|   Hooper|2018-04-21T12:49:...|         Wm|123456100|Main Street North|Main Street North...|
|   829|4210653310102868|Wethersfield|United States| EHolman@example.com|   1238933|        CT|   06109|      Etta|   Holman|2018-04-21T12:49:...|    Brendan|123453023|

root
 |-- _corrupt_record: string (nullable = true)

